# Mongodb

Dans ce TP nous allons utiliser la librairie pymongo, pour créer une base de données dans mongodb avec python puis la questionner.

### Data
Voici des données au format csv, elles sont une partie d'un dataset recensant les crimes au Etats-Unis entre 1984 et 2014.

https://drive.google.com/file/d/10z7kUXDO4BHffJ6ZfVc42CgIs5558vGd/view?usp=sharing

### Création de la BDD

In [12]:
import pandas as pd
from pymongo import MongoClient
import json
import streamlit as st

Créez une fonction python pour passer ce csv dans une base de données mongo, chaque lignes devra être un document.

### Requêter la BDD

Créez un connecteur pour votre BDD

In [28]:
client = MongoClient('localhost',
                    port=4344,
                    username='root',
                    password='root',
                    authMechanism='SCRAM-SHA-256')
db = client.crimes
table = db.documents

In [20]:
# Lecture du csv
df = pd.read_csv('Crime.csv')

In [29]:
# En une seule fois
if table.count_documents({}) == 0:    
    records = df.to_dict(orient='records')
    table.insert_many(records)

In [ ]:
# Ligne par ligne
if table.count_documents({}) == 0:
    json_docs = df.apply(lambda x: x.to_json(), axis=1)
    for document in json_docs:

        json_doc = json.loads(document)
        table.insert_one(json_doc)

##### Quelles sont les armes utilisées par les criminel?
Retournez par rapport à tous les documents les valeurs uniques de la clef weapon.

In [32]:
table.distinct("Weapon")

list(table.aggregate([
    { "$group": {
        "_id": "$Weapon", "count" : {"$sum": 1}
    }},
    { "$sort": {"count": -1}}
]))

[{'_id': 'Handgun', 'count': 45748},
 {'_id': 'Knife', 'count': 19586},
 {'_id': 'Blunt Object', 'count': 11150},
 {'_id': 'Shotgun', 'count': 7419},
 {'_id': 'Rifle', 'count': 4968},
 {'_id': 'Firearm', 'count': 3629},
 {'_id': 'Unknown', 'count': 3134},
 {'_id': 'Strangulation', 'count': 1742},
 {'_id': 'Fire', 'count': 1254},
 {'_id': 'Suffocation', 'count': 600},
 {'_id': 'Drowning', 'count': 265},
 {'_id': 'Gun', 'count': 212},
 {'_id': 'Drugs', 'count': 101},
 {'_id': 'Poison', 'count': 80},
 {'_id': 'Explosives', 'count': 61},
 {'_id': 'Fall', 'count': 50}]

##### Combien de crimes ont été commis en 1980?

In [ ]:
table.count_documents({"Year":1980})

##### Combien de crimes ont été commis par des hommes au Texas?

In [ ]:
table.count_documents({"Perpetrator Sex": "Male", "State": "Texas"})

##### Combien de crimes ont été commis par chaque sexe en Alaska?

In [ ]:
maleAndFemalePerpetratorsInAlaska = table.aggregate([
    { "$match": { "State": "Alaska" }},
    { "$group": {"_id": "$Perpetrator Sex", "crimes commis": {"$sum": 1}}},
])

list(maleAndFemalePerpetratorsInAlaska)

##### Combien y a-t'il eu de victimes dans chaque état?

In [ ]:
victimInEachState = table.aggregate([
    { "$group" : {
        "_id": "$State", "victimes": {"$sum": "$Victim Count"}
    }},
    { "$sort": { "victimes": -1}}
])

list(victimInEachState)

### Bonus

Installez la librairie streamlit
Créez un dashboard qui donne la posibilité de séléctionner une année pour retourner un barplot du nombre de crime commis avec chaque arme.

In [ ]:
victimWithEachWeapon = table.aggregate([
    {"$match": {"Year": 1980}},
    { "$group" : {
        "_id": "$Weapon", "victimes": {"$sum": "$Victim Count"}
    }}
])
    
list(victimWithEachWeapon)

In [ ]:
# QCM 2
victimWithEachAge = table.aggregate([
    { "$group" : {
        "_id": "$Victim Age", "victimes": {"$sum": 1}
    }},
    { "$sort" : {
        "victimes": -1
    }}
])
    
list(victimWithEachAge)



In [ ]:
# QCM 3
victimWithEachAgeByVictim = table.aggregate([
    {"$match": { "Weapon": "Drowning"}},
    { "$group" : {
        "_id": "$Victim Age", "victimes": {"$sum": 1}
    }},
    { "$sort" : {
        "victimes": -1
    }}
])
    
list(victimWithEachAgeByVictim)

In [ ]:
# QCM 4
victimWithEachAgeByVictim = table.aggregate([
    {"$match": { "Weapon": "Fall"}},
    { "$group" : {
        "_id": "$State", "victimes": {"$sum": 1}
    }},
    { "$sort" : {
        "victimes": -1
    }}
])
    
list(victimWithEachAgeByVictim)

In [ ]:
# QCM 5
victimWithEachAgeByVictim = table.aggregate([
    { "$group" : {
        "_id": "$State", "Moyenne": {"$avg": "$Victim Count"}
    }},
    { "$sort" : {
        "Moyenne": -1
    }}
])
    
list(victimWithEachAgeByVictim)